# ============================================================
# TASK 3 - FEATURE ENGINEERING & MODELLING (BCGX Forage - PowerCo churn)
# ============================================================

In [1]:
import pandas as pd
import numpy as np


## -------------------------
## 1) Load data + parse dates
## -------------------------

In [3]:
client_path = r"C:/Users/Utente/Desktop/BCG X (PER ME)/Task 2 Exploration Data Analysis (EDA)/client_data.csv"
price_path  = r"C:/Users/Utente/Desktop/BCG X (PER ME)/Task 2 Exploration Data Analysis (EDA)/price_data.csv"

client_data = pd.read_csv(client_path)
price_data  = pd.read_csv(price_path)

date_cols_client = ["date_activ", "date_end", "date_modif_prod", "date_renewal"]
for col in date_cols_client:
    if col in client_data.columns:
        client_data[col] = pd.to_datetime(client_data[col], errors="coerce")

price_data["price_date"] = pd.to_datetime(price_data["price_date"], errors="coerce")

print("client_data:", client_data.shape, "| price_data:", price_data.shape)


client_data: (14606, 26) | price_data: (193002, 8)


## -------------------------
## 2) Config
## -------------------------

In [4]:
T = pd.Timestamp("2015-12-31")
LEAK_COLS = ["dev_consum_no_churn", "dev_consum_with_log"]
USE_FORECAST = False  # nel modelling potrai fare ablation

def sanitize_inf(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for col in df.columns:
        if df[col].dtype != "object":
            df[col] = df[col].replace([np.inf, -np.inf], np.nan)
    return df

## ------------------------------------------------------------
## 3) Remove columns (costanti + leakage by construction + forecast_*)
## ------------------------------------------------------------

In [5]:
const_cols = [c for c in client_data.columns if client_data[c].nunique(dropna=False) <= 1]
if const_cols:
    client_data = client_data.drop(columns=const_cols)
    print("Dropped constant client cols:", const_cols)

leak_in_client = [c for c in LEAK_COLS if c in client_data.columns]
if leak_in_client:
    client_data = client_data.drop(columns=leak_in_client)
    print("Dropped leak cols from client_data:", leak_in_client)

if not USE_FORECAST:
    forecast_cols = [c for c in client_data.columns if c.startswith("forecast_")]
    if forecast_cols:
        client_data = client_data.drop(columns=forecast_cols)
        print("Dropped forecast cols:", len(forecast_cols))


Dropped forecast cols: 7


## ------------------------------------------------------------
## 4) Feature Engineering - price features (as-of T)
## ------------------------------------------------------------

In [6]:
PRICE_VARS = [
    "price_off_peak_var", "price_peak_var", "price_mid_peak_var",
    "price_off_peak_fix", "price_peak_fix", "price_mid_peak_fix",
]

p = price_data.dropna(subset=["price_date"]).copy()
p = p[p["price_date"] <= T].sort_values(["id", "price_date"])

def make_price_features(df_id: pd.DataFrame) -> pd.Series:
    df_id = df_id.sort_values("price_date")
    last = df_id.iloc[-1]
    out = {}

    for v in PRICE_VARS:
        out[f"{v}_last"] = last[v]
        out[f"{v}_mean"] = df_id[v].mean()
        out[f"{v}_std"]  = df_id[v].std()

        y = pd.to_numeric(df_id[v], errors="coerce").to_numpy()
        x = np.arange(len(y), dtype=float)
        if len(y) >= 3 and np.isfinite(y).sum() >= 3:
            out[f"{v}_slope"] = np.polyfit(x, y, 1)[0]
        else:
            out[f"{v}_slope"] = np.nan

    out["var_peak_minus_offpeak_last"] = last["price_peak_var"] - last["price_off_peak_var"]
    out["fix_peak_minus_offpeak_last"] = last["price_peak_fix"] - last["price_off_peak_fix"]
    out["var_peak_over_offpeak_last"]  = (
        (last["price_peak_var"] / last["price_off_peak_var"])
        if pd.notna(last["price_off_peak_var"]) and last["price_off_peak_var"] != 0
        else np.nan
    )
    return pd.Series(out)

try:
    price_feat = p.groupby("id").apply(make_price_features, include_groups=False).reset_index()
except TypeError:
    price_feat = p.groupby("id").apply(make_price_features).reset_index()

print("price_feat:", price_feat.shape)


price_feat: (16096, 28)


## ------------------------------------------------------------
## 5) Feature Engineering - time-derived client features (as-of T)
## ------------------------------------------------------------

In [7]:
c = client_data.copy()

c["tenure_days"] = (T - c["date_activ"]).dt.days
c["days_since_prod_modif"] = (T - c["date_modif_prod"]).dt.days
c["days_to_renewal"] = (c["date_renewal"] - T).dt.days

# SAFE dev_consum for everyone (NOT conditional on churn)
c["dev_consum"] = c["cons_last_month"] - (c["cons_12m"] / 12.0)

# In Task 4 droppiamo le date raw; qui le teniamo o le droppiamo: scegliamo di dropparle già ora.
date_cols_present = [col for col in date_cols_client if col in c.columns]
c = c.drop(columns=date_cols_present)

## ------------------------------------------------------------
## 6) Combine datasets (merge finale) -> df feature-ready
## ------------------------------------------------------------

In [8]:
df = c.merge(price_feat, on="id", how="left")
df = sanitize_inf(df)

print("df feature-ready:", df.shape)

df feature-ready: (14606, 46)


## ------------------------------------------------------------
## 7) Sanity checks finali (ready to model)
## ------------------------------------------------------------

In [9]:
print("\n--- SANITY CHECKS ---")
print(df.head(3))
print("\nMissing rate (top 15):")
print(df.isna().mean().sort_values(ascending=False).head(15))


--- SANITY CHECKS ---
                                 id                     channel_sales  \
0  24011ae4ebbe3035111d65fa7c15bc57  foosdfpfkusacimwkcsosbicdxkicaua   
1  d29c2c54acc38ff3c0614d0a653813dd                           MISSING   
2  764c75f661154dac3a6c254cd082ea7d  foosdfpfkusacimwkcsosbicdxkicaua   

   cons_12m  cons_gas_12m  cons_last_month has_gas  imp_cons  \
0         0         54946                0       t       0.0   
1      4660             0                0       f       0.0   
2       544             0                0       f       0.0   

   margin_gross_pow_ele  margin_net_pow_ele  nb_prod_act  ...  \
0                 25.44               25.44            2  ...   
1                 16.38               16.38            1  ...   
2                 28.60               28.60            1  ...   

   price_peak_fix_mean  price_peak_fix_std price_peak_fix_slope  \
0             22.35201            7.039226            -0.927593   
1              0.00000          

# ============================================================
# TASK 4 - MODELLING
# ============================================================

In [10]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.inspection import permutation_importance


## -------------------------
## Config modelling
## -------------------------

In [11]:
RANDOM_STATE = 42
TEST_SIZE = 0.25
TOPK_FRAC = 0.10

## -------------------------
## Helper: precision/recall@k
## -------------------------

In [12]:
def precision_recall_at_k(y_true, y_score, k_frac=0.10):
    y_true = pd.Series(y_true).reset_index(drop=True)
    y_score = pd.Series(y_score).reset_index(drop=True)
    n = len(y_true)
    k = max(1, int(n * k_frac))
    idx = np.argsort(-y_score)[:k]
    precision = y_true.iloc[idx].mean()
    recall = y_true.iloc[idx].sum() / y_true.sum()
    return float(precision), float(recall), k

## -------------------------
## Helper: preprocess builders
## -------------------------

In [13]:
def build_preprocess(X):
    cat_cols = [c for c in X.columns if X[c].dtype == "object"]
    num_cols = [c for c in X.columns if c not in cat_cols]
    return ColumnTransformer(
        transformers=[
            ("num", Pipeline([("imp", SimpleImputer(strategy="median"))]), num_cols),
            ("cat", Pipeline([
                ("imp", SimpleImputer(strategy="most_frequent")),
                ("oh", OneHotEncoder(handle_unknown="ignore"))
            ]), cat_cols),
        ]
    )

def build_preprocess_lr(X):
    cat_cols = [c for c in X.columns if X[c].dtype == "object"]
    num_cols = [c for c in X.columns if c not in cat_cols]
    return ColumnTransformer(
        transformers=[
            ("num", Pipeline([
                ("imp", SimpleImputer(strategy="median")),
                ("scaler", StandardScaler(with_mean=False))
            ]), num_cols),
            ("cat", Pipeline([
                ("imp", SimpleImputer(strategy="most_frequent")),
                ("oh", OneHotEncoder(handle_unknown="ignore"))
            ]), cat_cols),
        ]
    )

## -------------------------
## Costruzione X,y (da df feature-ready)
## -------------------------

In [14]:
y = df["churn"].astype(int)

exclude = set(["id", "churn"])
if not USE_FORECAST:
    exclude |= set([c for c in df.columns if c.startswith("forecast_")])

X = df[[c for c in df.columns if c not in exclude]].copy()
X = X.drop(columns=[c for c in LEAK_COLS if c in X.columns], errors="ignore")
X = sanitize_inf(X)

print("X:", X.shape, "| churn rate:", float(y.mean()))

X: (14606, 44) | churn rate: 0.09715185540188963


## -------------------------
## Log-transform (solo su X_log)
## -------------------------

In [15]:
LOG_COLS = ["cons_12m","cons_gas_12m","cons_last_month","imp_cons",
            "margin_gross_pow_ele","margin_net_pow_ele","pow_max","dev_consum"]

LOG_COLS = [c for c in LOG_COLS if c in X.columns]

X_log = X.copy()
for col in LOG_COLS:
    X_log[col] = np.log1p(pd.to_numeric(X_log[col], errors="coerce").clip(lower=0))

# NB: NON logghiamo net_margin perché serve raw nel business case.
# Se vuoi loggarlo per il modello, crea net_margin_log separato:
if "net_margin" in X_log.columns:
    X_log["net_margin_log"] = np.log1p(pd.to_numeric(X_log["net_margin"], errors="coerce").clip(lower=0))

## -------------------------
## Train/eval function
## -------------------------

In [21]:
def train_eval_models(X_in, y_in, random_state=42, test_size=0.25, topk_frac=0.10):
    X_train, X_test, y_train, y_test = train_test_split(
        X_in, y_in, test_size=test_size, random_state=random_state, stratify=y_in
    )

    preprocess_rf = build_preprocess(X_train)
    preprocess_lr = build_preprocess_lr(X_train)

    logreg = Pipeline(steps=[
        ("prep", preprocess_lr),
        ("clf", LogisticRegression(
            max_iter=5000,
            class_weight="balanced",
            solver="saga",
            n_jobs=-1
        ))
    ])

    rf = Pipeline(steps=[
        ("prep", preprocess_rf),
        ("clf", RandomForestClassifier(
            n_estimators=400,
            random_state=random_state,
            class_weight="balanced_subsample",
            n_jobs=-1,
            min_samples_leaf=5,
            max_features=0.5
        ))
    ])

    logreg.fit(X_train, y_train)
    rf.fit(X_train, y_train)

    proba_lr = logreg.predict_proba(X_test)[:, 1]
    proba_rf = rf.predict_proba(X_test)[:, 1]

    ap_lr = average_precision_score(y_test, proba_lr)
    ap_rf = average_precision_score(y_test, proba_rf)

    p_lr, r_lr, _ = precision_recall_at_k(y_test, proba_lr, topk_frac)
    p_rf, r_rf, _ = precision_recall_at_k(y_test, proba_rf, topk_frac)

    return {
        "logreg": {"ap": float(ap_lr), "p10": p_lr, "r10": r_lr},
        "rf":     {"ap": float(ap_rf), "p10": p_rf, "r10": r_rf},
        "models": {"logreg": logreg, "rf": rf},
        "split":  {"X_test": X_test, "y_test": y_test},
    }

## -------------------------
## Baseline / No-Price / Bill pressure
## -------------------------

In [22]:
base = train_eval_models(X_log, y, random_state=RANDOM_STATE, test_size=TEST_SIZE, topk_frac=TOPK_FRAC)
print("\nBASELINE\nLogReg:", base["logreg"], "\nRF   :", base["rf"])

price_cols = [c for c in X_log.columns if c.startswith("price_") or "peak_minus" in c or "peak_over" in c]
X_noprice = X_log.drop(columns=price_cols, errors="ignore")
nop = train_eval_models(X_noprice, y, random_state=RANDOM_STATE, test_size=TEST_SIZE, topk_frac=TOPK_FRAC)
print("\nNO-PRICE\nLogReg:", nop["logreg"], "\nRF   :", nop["rf"])

X_bill = X_log.copy()
# bill pressure (usa consumi/potenza RAW o log? meglio RAW)
# Qui usiamo le colonne RAW originali (da X) per coerenza economica:
if "price_off_peak_var_last" in X_bill.columns and "cons_12m" in X.columns:
    X_bill["bill_var_off_last_x_cons12m"] = X_bill["price_off_peak_var_last"] * X["cons_12m"]
if "price_peak_var_last" in X_bill.columns and "cons_12m" in X.columns:
    X_bill["bill_var_peak_last_x_cons12m"] = X_bill["price_peak_var_last"] * X["cons_12m"]
if "price_off_peak_fix_last" in X_bill.columns and "pow_max" in X.columns:
    X_bill["bill_fix_off_last_x_pow"] = X_bill["price_off_peak_fix_last"] * X["pow_max"]

X_bill = sanitize_inf(X_bill)

bil = train_eval_models(X_bill, y, random_state=RANDOM_STATE, test_size=TEST_SIZE, topk_frac=TOPK_FRAC)
print("\nWITH BILL PRESSURE\nLogReg:", bil["logreg"], "\nRF   :", bil["rf"])



BASELINE
LogReg: {'ap': 0.16989527046610545, 'p10': 0.23013698630136986, 'r10': 0.23661971830985915} 
RF   : {'ap': 0.3273387622568088, 'p10': 0.3424657534246575, 'r10': 0.352112676056338}

NO-PRICE
LogReg: {'ap': 0.16381251776475558, 'p10': 0.2136986301369863, 'r10': 0.21971830985915494} 
RF   : {'ap': 0.3233785961880004, 'p10': 0.33972602739726027, 'r10': 0.3492957746478873}

WITH BILL PRESSURE
LogReg: {'ap': 0.17092991253327736, 'p10': 0.20821917808219179, 'r10': 0.2140845070422535} 
RF   : {'ap': 0.33260253643101256, 'p10': 0.336986301369863, 'r10': 0.3464788732394366}


## -------------------------
## CV comparativa (RF)
## -------------------------

In [23]:
def make_rf_pipeline(Xtr):
    preprocess = build_preprocess(Xtr)
    return Pipeline(steps=[
        ("prep", preprocess),
        ("clf", RandomForestClassifier(
            n_estimators=400,
            random_state=RANDOM_STATE,
            class_weight="balanced_subsample",
            n_jobs=-1,
            min_samples_leaf=5,
            max_features=0.5
        ))
    ])

def cv_compare_sets(feature_sets: dict, y_in, n_splits=5, random_state=42, topk_frac=0.10):
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    rows = []
    for name, Xset in feature_sets.items():
        Xset = sanitize_inf(Xset)
        aps, pks, rks = [], [], []
        for tr, te in cv.split(Xset, y_in):
            Xtr, Xte = Xset.iloc[tr], Xset.iloc[te]
            ytr, yte = y_in.iloc[tr], y_in.iloc[te]
            model = make_rf_pipeline(Xtr)
            model.fit(Xtr, ytr)
            proba = model.predict_proba(Xte)[:, 1]
            aps.append(average_precision_score(yte, proba))
            p_k, r_k, _ = precision_recall_at_k(yte, proba, topk_frac)
            pks.append(p_k); rks.append(r_k)
        rows.append({
            "set": name,
            "AP_mean": float(np.mean(aps)),
            "AP_std": float(np.std(aps)),
            "P@k_mean": float(np.mean(pks)),
            "R@k_mean": float(np.mean(rks)),
        })
    return pd.DataFrame(rows).sort_values("AP_mean", ascending=False)

feature_sets = {"baseline": X_log, "no_price": X_noprice, "bill_pressure": X_bill}
cv_table = cv_compare_sets(feature_sets, y, n_splits=5, random_state=RANDOM_STATE, topk_frac=TOPK_FRAC)
print("\nCV TABLE")
print(cv_table)


CV TABLE
             set   AP_mean    AP_std  P@k_mean  R@k_mean
2  bill_pressure  0.321534  0.008754  0.314384  0.323466
0       baseline  0.316913  0.008343  0.313699  0.322762
1       no_price  0.304349  0.008679  0.309589  0.318529


## -------------------------
## Altri parametri
## -------------------------

In [19]:
from itertools import product

def rf_cv_tiny_grid(X_in, y_in, n_splits=5, random_state=42, k_frac=0.20):
    X_in = sanitize_inf(X_in)
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    grid = {
        "min_samples_leaf": [1, 5, 10],
        "max_features": ["sqrt", 0.5],
    }

    rows = []
    for msl, mf in product(grid["min_samples_leaf"], grid["max_features"]):
        aps, pks, rks = [], [], []

        for tr, te in cv.split(X_in, y_in):
            Xtr, Xte = X_in.iloc[tr], X_in.iloc[te]
            ytr, yte = y_in.iloc[tr], y_in.iloc[te]

            preprocess = build_preprocess(Xtr)
            model = Pipeline(steps=[
                ("prep", preprocess),
                ("clf", RandomForestClassifier(
                    n_estimators=400,
                    random_state=random_state,
                    class_weight="balanced_subsample",
                    n_jobs=-1,
                    min_samples_leaf=msl,
                    max_features=mf
                ))
            ])

            model.fit(Xtr, ytr)
            proba = model.predict_proba(Xte)[:, 1]

            aps.append(average_precision_score(yte, proba))
            p_k, r_k, _ = precision_recall_at_k(yte, proba, k_frac)
            pks.append(p_k); rks.append(r_k)

        rows.append({
            "min_samples_leaf": msl,
            "max_features": mf,
            "AP_mean": float(np.mean(aps)),
            "AP_std": float(np.std(aps)),
            "P@k_mean": float(np.mean(pks)),
            "R@k_mean": float(np.mean(rks)),
        })

    return pd.DataFrame(rows).sort_values("AP_mean", ascending=False)

# Esempio: tuning sul set finale
grid_res = rf_cv_tiny_grid(X_bill, y, n_splits=5, random_state=RANDOM_STATE, k_frac=0.20)
grid_res.head(10)

,min_samples_leaf,max_features,AP_mean,AP_std,P@k_mean,R@k_mean
1,1,0.5,0.323870,0.009957,0.223973,0.460887
3,5,0.5,0.321534,0.008754,0.228425,0.470054
0,1,sqrt,0.310885,0.010506,0.222945,0.458762
5,10,0.5,0.310008,0.011682,0.232877,0.479222
2,5,sqrt,0.296712,0.005146,0.225000,0.463019
4,10,sqrt,0.280068,0.011109,0.223288,0.459488


## -------------------------
## Final model + lift table
## -------------------------

In [24]:
def lift_table(y_true, y_score, fracs=(0.01,0.02,0.05,0.10,0.15,0.20,0.30)):
    base_rate = float(pd.Series(y_true).mean())
    rows = []
    for f in fracs:
        p, r, k = precision_recall_at_k(y_true, y_score, f)
        rows.append({
            "contact_frac": f, "k": k,
            "precision": p, "recall": r,
            "lift_vs_base": (p / base_rate) if base_rate > 0 else np.nan
        })
    return pd.DataFrame(rows)

X_final = X_bill
X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

final_model = make_rf_pipeline(X_train)
final_model.fit(X_train, y_train)
proba_test = final_model.predict_proba(X_test)[:, 1]

lift = lift_table(y_test, proba_test)
print("\nLIFT TABLE (test)")
print(lift)


LIFT TABLE (test)
   contact_frac     k  precision    recall  lift_vs_base
0          0.01    36   0.861111  0.087324      8.858529
1          0.02    73   0.671233  0.138028      6.905190
2          0.05   182   0.483516  0.247887      4.974091
3          0.10   365   0.336986  0.346479      3.466687
4          0.15   547   0.277879  0.428169      2.858635
5          0.20   730   0.231507  0.476056      2.381586
6          0.30  1095   0.196347  0.605634      2.019886


## -------------------------
## Permutation importance
## -------------------------

In [25]:
pi_final = permutation_importance(
    final_model, X_test, y_test,
    n_repeats=10, random_state=RANDOM_STATE,
    scoring="average_precision"
)

imp_final = pd.DataFrame({
    "feature": X_test.columns,
    "perm_importance": pi_final.importances_mean
}).sort_values("perm_importance", ascending=False)

print("\nTOP 20 PERMUTATION IMPORTANCE")
print(imp_final.head(20))


TOP 20 PERMUTATION IMPORTANCE
                        feature  perm_importance
7            margin_net_pow_ele         0.120303
6          margin_gross_pow_ele         0.067551
15              days_to_renewal         0.067367
11                    origin_up         0.035805
13                  tenure_days         0.021623
14        days_since_prod_modif         0.015549
47      bill_fix_off_last_x_pow         0.013943
16                   dev_consum         0.011537
9                    net_margin         0.010241
44               net_margin_log         0.010001
3               cons_last_month         0.008952
12                      pow_max         0.006618
45  bill_var_off_last_x_cons12m         0.006212
0                 channel_sales         0.005867
1                      cons_12m         0.005637
20     price_off_peak_var_slope         0.003938
19       price_off_peak_var_std         0.003771
32     price_off_peak_fix_slope         0.002912
31       price_off_peak_fix_std       

## -------------------------
## CV robusta (manuale, P@20 e R@20)
## -------------------------

In [26]:
def cv_ap_and_recallk_manual(X_in, y_in, k_frac=0.20, n_splits=5, random_state=42):
    X_in = sanitize_inf(X_in)
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    aps, pks, rks = [], [], []

    for fold, (tr, te) in enumerate(cv.split(X_in, y_in), 1):
        Xtr, Xte = X_in.iloc[tr], X_in.iloc[te]
        ytr, yte = y_in.iloc[tr], y_in.iloc[te]

        model = make_rf_pipeline(Xtr)
        model.fit(Xtr, ytr)
        proba = model.predict_proba(Xte)[:, 1]

        ap = average_precision_score(yte, proba)
        p_k, r_k, _ = precision_recall_at_k(yte, proba, k_frac)

        aps.append(ap); pks.append(p_k); rks.append(r_k)
        print(f"Fold {fold}: AP={ap:.4f} | P@{int(k_frac*100)}%={p_k:.3f} | R@{int(k_frac*100)}%={r_k:.3f}")

    return {
        "AP_mean": float(np.mean(aps)),
        "AP_std": float(np.std(aps)),
        "P@k_mean": float(np.mean(pks)),
        "R@k_mean": float(np.mean(rks)),
    }

print("\nCV ROBUSTA (bill_pressure, k=20%)")
cv_stats = cv_ap_and_recallk_manual(X_final, y, k_frac=0.20, n_splits=5, random_state=RANDOM_STATE)
print(cv_stats)


CV ROBUSTA (bill_pressure, k=20%)
Fold 1: AP=0.3223 | P@20%=0.238 | R@20%=0.489
Fold 2: AP=0.3314 | P@20%=0.238 | R@20%=0.489
Fold 3: AP=0.3288 | P@20%=0.224 | R@20%=0.461
Fold 4: AP=0.3066 | P@20%=0.211 | R@20%=0.433
Fold 5: AP=0.3186 | P@20%=0.231 | R@20%=0.477
{'AP_mean': 0.3215341131472756, 'AP_std': 0.008754291779871103, 'P@k_mean': 0.22842465753424662, 'R@k_mean': 0.47005424774797194}


## -------------------------
## Business case (policy a due livelli) - usa net_margin RAW da df
## -------------------------

In [27]:
q = df["net_margin"].quantile([0.99]).to_dict()
CAP = float(q[0.99])

K_FRAC = 0.20
ALPHA = 0.3
UPLIFT_ASSUMED = 0.10
LOW_COST_CONTACT = 0.2
OFFER_COST = 2.0

idx = X_test.index
d = df.loc[idx, ["net_margin", "churn"]].copy()

d["net_margin_cap"] = d["net_margin"].clip(lower=0, upper=CAP)
d["p_churn"] = proba_test
d["residual_value"] = ALPHA * d["net_margin_cap"]

k = max(1, int(len(d) * K_FRAC))
sel = np.argsort(-d["p_churn"].values)[:k]
tgt2 = d.iloc[sel].copy()

tgt2["max_total_cost_break_even"] = tgt2["p_churn"] * UPLIFT_ASSUMED * tgt2["residual_value"]
tgt2["offer_recommended"] = tgt2["max_total_cost_break_even"] >= (LOW_COST_CONTACT + OFFER_COST)

offer_count = int(tgt2["offer_recommended"].sum())
total_target = len(tgt2)

expected_churn_in_target = float(tgt2["p_churn"].sum())
expected_churn_prevented = expected_churn_in_target * UPLIFT_ASSUMED

total_cost = (total_target * LOW_COST_CONTACT) + (offer_count * OFFER_COST)
total_benefit = float((tgt2["p_churn"] * UPLIFT_ASSUMED * tgt2["residual_value"]).sum())

print("\nBUSINESS CASE (policy)")
print("Top-20% targeted:", total_target)
print("Offer recommended count:", offer_count, f"({offer_count/total_target:.2%})")
print("Expected churn prevented (uplift):", expected_churn_prevented)
print("Total cost (policy):", total_cost)
print("Total benefit (proxy):", total_benefit)
print("Net expected value:", total_benefit - total_cost)


BUSINESS CASE (policy)
Top-20% targeted: 730
Offer recommended count: 243 (33.29%)
Expected churn prevented (uplift): 27.3692243125151
Total cost (policy): 632.0
Total benefit (proxy): 1607.224330688525
Net expected value: 975.224330688525


In [28]:
thr = LOW_COST_CONTACT + OFFER_COST

print("Threshold (contact+offer):", thr)
print("Offer rate:", tgt2["offer_recommended"].mean())

# Diagnosi delle leve
print("\n--- TARGET SUMMARY (top-20%) ---")
print("p_churn mean:", tgt2["p_churn"].mean())
print("p_churn median:", tgt2["p_churn"].median())
print("net_margin_cap mean:", tgt2["net_margin_cap"].mean())
print("net_margin_cap median:", tgt2["net_margin_cap"].median())
print("residual_value mean:", tgt2["residual_value"].mean())

# Quanto manca in media ai NON raccomandati per diventarlo?
gap = thr - tgt2.loc[~tgt2["offer_recommended"], "max_total_cost_break_even"]
print("\nAvg gap to threshold (non-recommended):", gap.mean())

# Distribuzione break-even vs soglia
print("\nmax_total_cost_break_even quantiles:")
print(tgt2["max_total_cost_break_even"].quantile([0.1,0.25,0.5,0.75,0.9,0.95,0.99]))

Threshold (contact+offer): 2.2
Offer rate: 0.33287671232876714

--- TARGET SUMMARY (top-20%) ---
p_churn mean: 0.3749208809933575
p_churn median: 0.33973042061037806
net_margin_cap mean: 197.01063493150684
net_margin_cap median: 124.535
residual_value mean: 59.10319047945205

Avg gap to threshold (non-recommended): 1.3072663448154918

max_total_cost_break_even quantiles:
0.10     0.249226
0.25     0.594694
0.50     1.343122
0.75     2.909904
0.90     5.669452
0.95     7.124058
0.99    10.634786
Name: max_total_cost_break_even, dtype: float64


In [29]:
# Confronto recommended vs non
grp = tgt2.groupby("offer_recommended")[["p_churn","net_margin_cap","residual_value","max_total_cost_break_even"]].mean()
print(grp)

                    p_churn  net_margin_cap  residual_value  \
offer_recommended                                             
False              0.360178       86.520000        25.95600   
True               0.404468      418.446599       125.53398   

                   max_total_cost_break_even  
offer_recommended                             
False                               0.892734  
True                                4.824951  


In [30]:
tgt2.groupby("offer_recommended")["churn"].mean()

offer_recommended
False    0.211499
True     0.271605
Name: churn, dtype: float64

In [31]:
P_MIN = 0.30  # esempio, da tarare
tgt2["offer_recommended_v2"] = (
    (tgt2["max_total_cost_break_even"] >= (LOW_COST_CONTACT + OFFER_COST)) &
    (tgt2["p_churn"] >= P_MIN)
)
tgt2["offer_recommended_v2"].mean(), tgt2.groupby("offer_recommended_v2")["churn"].mean()

(np.float64(0.2602739726027397),
 offer_recommended_v2
 False    0.200000
 True     0.321053
 Name: churn, dtype: float64)

In [34]:
UPLIFT_EMAIL = 0.02        # esempio: uplift piccolo da contatto
UPLIFT_OFFER_INC = 0.08    # esempio: uplift addizionale dell'incentivo
# (totale offer = 0.02 + 0.08 = 0.10)

def eval_policy_fixed(tgt, flag_col):
    offer_mask = tgt[flag_col].astype(bool)

    # costi
    total_target = len(tgt)
    offer_count = int(offer_mask.sum())
    total_cost = (total_target * LOW_COST_CONTACT) + (offer_count * OFFER_COST)

    # benefici: email su tutti + incentivo solo su recommended
    benefit_email = float((tgt["p_churn"] * UPLIFT_EMAIL * tgt["residual_value"]).sum())
    benefit_offer = float((tgt.loc[offer_mask, "p_churn"] * UPLIFT_OFFER_INC * tgt.loc[offer_mask, "residual_value"]).sum())
    total_benefit = benefit_email + benefit_offer
    net_ev = total_benefit - total_cost

    return {
        "offer_rate": offer_count / total_target,
        "offer_count": offer_count,
        "total_target": total_target,
        "total_cost": total_cost,
        "benefit_email": benefit_email,
        "benefit_offer": benefit_offer,
        "total_benefit": total_benefit,
        "net_ev": net_ev,
        "churn_true": float(tgt.loc[offer_mask, "churn"].mean()) if offer_count > 0 else np.nan,
        "churn_false": float(tgt.loc[~offer_mask, "churn"].mean()) if offer_count < total_target else np.nan,
    }

res_v1 = eval_policy_fixed(tgt2, "offer_recommended")
res_v2 = eval_policy_fixed(tgt2, "offer_recommended_v2")

pd.DataFrame([res_v1, res_v2], index=["v1_break_even", "v2_break_even+pmin"])

,offer_rate,offer_count,total_target,total_cost,benefit_email,benefit_offer,total_benefit,net_ev,churn_true,churn_false
v1_break_even,0.332877,243,730,632.0,321.444866,937.970432,1259.415299,627.415299,0.271605,0.211499
v2_break_even+pmin,0.260274,190,730,526.0,321.444866,763.200594,1084.645460,558.645460,0.321053,0.200000


In [35]:
delta_ev = 627.41529862864 - 558.645459755033
delta_offers = 243 - 190
delta_ev_per_extra_offer = delta_ev / delta_offers
delta_ev, delta_offers, delta_ev_per_extra_offer

(68.769838873607, 53, 1.2975441296906982)

## -----------------------------------------------------------
# Conclusione del notebook
## -----------------------------------------------------------
In questo notebook abbiamo costruito un dataset “snapshot” **as-of T = 31/12/2015**, creando feature **senza leakage**: aggregazioni dei prezzi fino a T (last/mean/std/slope per ciascuna tariffa) e feature temporali lato cliente (tenure, giorni da modifica prodotto, giorni al rinnovo) più una proxy di deviazione dei consumi (dev_consum) calcolata per tutti i clienti. Il merge finale produce un dataset feature-ready di 14.606 righe (clienti) con feature numeriche/categoriche e variabili prezzo aggregate.

Sul modelling abbiamo confrontato tre set di feature (**baseline, no_price, bill_pressure**) con metriche adatte a class imbalance (churn rate ≈**9.7%**) e validazione cross-validation. La Random Forest è stata leggermente regolarizzata tramite tuning minimale (scelta finale: min_samples_leaf=5, max_features=0.5, n_estimators=400, class_weight="balanced_subsample"), privilegiando stabilità e performance operative sul top-k. Il set **bill_pressure** (proxy dell’impatto economico: prezzo×consumo/potenza) risulta il migliore e stabile: **CV AP_mean ≈ 0.3215** (std ≈ 0.0088), superiore a baseline (≈ 0.3169) e no_price (≈ 0.3043).

In ottica operativa, adottiamo una strategia **top-k**: sul test set, contattando il **top-20%** (730 clienti) otteniamo **precision ≈ 0.2315, recall ≈ 0.4761** (lift ≈ 2.38×), cioè quasi metà dei churner è intercettata concentrando l’azione su un quinto della base.

Per la parte di business case abbiamo trasformato il ranking in una policy a due livelli: **email low-cost a tutto il top-20%** e **micro-incentivo selettivo** solo dove conviene economicamente. La selezione dell’offerta è guidata soprattutto dal valore cliente (net margin cappato al 99° percentile), e infatti i clienti “offer recommended” hanno churn osservato più alto (≈ 27.2% vs 21.1%) e valore medio molto superiore. Come raffinamento, l’aggiunta di una soglia minima di rischio (p_churn ≥ 0.30) aumenta la “purezza” del gruppo incentivato (≈ 32.1% churn osservato) riducendo il numero di incentivi. Nel confronto scenario-based con uplift separato (es. email 2% + incentivo addizionale 8%), la policy **v1 (solo break-even**)** massimizza l’EV atteso (≈ **+627**) ma richiede ~33% incentivi nel target; la policy **v2 (break-even + soglia rischio)** riduce gli incentivi (~26%) ma produce EV totale inferiore (≈ **+559**). La scelta finale dipende da vincoli di budget/capacità: senza vincoli espliciti v1 è preferibile, mentre v2 è una valida alternativa più parsimoniosa.

***(Nota metodologica: la ROI resta scenario-based e dipende da uplift e dalla calibrazione delle probabilità; in produzione andrebbe validata con A/B test e, se necessario, con calibrazione delle proba.)***